<a href="https://colab.research.google.com/github/dmejial/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AUTOENCODERS_BOOSTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CONFIG

conf_sample

In [0]:
# HEADER FILE #
from consensus.policy import combine_weighted
from torch.nn import MSELoss
from layer_decays.decay import LinearDecay

######################################################
__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'
######################################################
# when constructing an AE, it indicates whether
# you should use an odd number of layers or not
# BOTTLENECK == True -> odd 
# BOTTLENECK == False -> even
BOTTLENECK = True
# The number of training epochs for each 
# iteration
EPOCHS = 50
# indicates the maximum of encoder layers to train
# the model
MAX_ENC_LAYERS = 4
# The number of folds for which we repeat the
# experiments
FOLDS = 30
# The maximum number of iterations
ITERATIONS = 20
# The loss optimisation function to use
loss = MSELoss()
# Consensus policy to adopt when combining the 
# reconstruction errors of each data point
consensus = combine_weighted
# shrinker that calculates the neurons
# of the layers in the autoencoder
shrinker = LinearDecay(0.5)
# stop training threshold
STOP_TH = 1e-4
######################################################
# Common Optimiser Parameters
# Learning rate
LR = 1e-3
WEIGHT_DECAY = 1e-5
######################################################
# Flag that determines the data normalisation
NORM = True
# The value at the beginning of the normalisation
# interval
B_NORM = 0.0
# The value at the end of the normalisation interval
E_NORM = 1.0
# Data Path
DPATH = '../resources/data/'
# The list of datasets to read and train on
DATASETS = ['Lympho.csv']
######################################################


##CONSENSUS

 policy

In [0]:
import numpy as np
import powerlaw
from collections import Counter
from scipy.stats import entropy

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'

def combine_sum(matrix):
    return np.sum(matrix, axis=0)

def combine_last(matrix):
    return np.sum(matrix[-1], axis=0)

def combine_best(matrix):
    # get the row-wise sum of rec errors
    s = np.sum(matrix, axis=1)

    # get the first index of the component with the 
    # lowest rec error
    min_rec_i = np.where(s == np.amin(s))[0][0]
    # return all the rec errors of the data points
    # in the min_rec_i-th row of the original matrix
    return matrix[min_rec_i]

def combine_fromsecond(matrix):
    return np.sum(matrix[1:], axis=0)

def combine_weighted(matrix):
    # do not consider the first iteration of BAE
    num = np.sum(matrix[1:], axis=1)
    weights = (1 / num) / np.sum(1 / num)

    return np.sum(matrix[1:] * weights[:, np.newaxis],\
        axis=0)

def combine_kl_divergence(matrix):
    # discard the first component from the ensemble
    matrix = matrix[1:]

    weights = list()
    # loop through all the reconstruction errors
    for rec_errors in matrix:
        # fit a power law of the rec errors
        res = powerlaw.Fit(rec_errors,\
        xmin=rec_errors.min(), xmax=rec_errors.max(),\
            discrete=True)
        # get the occurrence probability of each
        # data point
        _, prob = res.ccdf()
        # generate our own distribution of data
        y = np.array(list(Counter(rec_errors).values()))
        # scale it to [0,1]
        y = y/y.max()
        # calculate the kl-divergence between
        # our distribution and the power law
        score = entropy(y, prob)
        # append the kl-divergence to the
        # list of weights
        weights.append(score)

    weights = np.array(weights)

    return np.sum(matrix * weights[:, np.newaxis], axis=0)



##LAYER_DECAY

 decay

In [0]:
import math
from abc import ABC, abstractmethod

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'

""" 
    This class represents the decay ratio from one
    layer to the next in the DeepSymAE.
"""
class DecayAB(ABC):
 
    def __init__(self, val):
        self.val = val
        super().__init__()
    
    @abstractmethod
    def shrink(self, l, n_d):
        return self.val * l * n_d

class LinearDecay(DecayAB):
    
    def shrink(self, l, n_d):
        return math.floor(self.val * n_d)

class ExpDecay(DecayAB):

    def shrink(self, l, n_d):
        return math.ceil(n_d * math.exp(-(self.val * l)))

class ConstantDecay(DecayAB):

    def shrink(self, l, n_d):
        return self.val[l]

class CompositeDecay():
    def __init__(self, shrinkers, choices, size):
        self.shrinkers = shrinkers
        self.choices = choices + [0]
        self.size = size

    def shrink(self, l, n_d):
        for i in range(len(self.choices)):
            if n_d/self.size >= self.choices[i]:
                return self.shrinkers[i].shrink(l, n_d)


##MODELS

fcae

In [0]:
import torch
import torch.nn as nn
import copy
import numpy as np

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'

class FullyConnectedAE(nn.Module):

    def __init__(self, enc_layers, dim,\
        shrinker, bottleneck=True, _min=3,
        device='cpu'):

        super(FullyConnectedAE, self).__init__()

        self.enc_layers = enc_layers
        self.dim = dim
        self.min = _min
        self.bottleneck = bottleneck
        self.shrinker = shrinker
        self.device = device

        self.build()

    def build(self):
        layers = []
        sizes = [self.dim]
        
        #Encoder Layers
        for l in range(self.enc_layers):
            n_h = max([self.shrinker.shrink(l+1,\
                sizes[-1]), self.min])

            layers.append(nn.Linear(sizes[-1], n_h).to(self.device) )
            # add a sigmoid layer as the first 
            # activation function
            if l == 0:
                layers.append(nn.Sigmoid().to(self.device))
            else: # the other activation layers are ReLU
                layers.append(nn.ReLU(inplace=False).to(self.device))

            sizes.append(n_h)

            if n_h == 1 or self.enc_layers == -1:
                break

        #Decoder Layers
        for l in range(self.enc_layers, 1, -1):
            layers.append(nn.Linear(sizes[l], sizes[l-1]).to(self.device))
            layers.append(nn.ReLU(inplace=False).to(self.device))
        
        layers.append(nn.Linear(sizes[1], sizes[0]).to(self.device))
        layers.append(nn.Sigmoid().to(self.device))


        # if this flag is set then we need an
        # odd number of layers
        # the last layers is going to be placed at
        # the centre of the model. Hence, the
        # bottleneck naming. Because an
        # autoencoder is symmetric, the bottleneck's
        # input and output size are equal to the last
        # encoder layer's output size.
        if self.bottleneck == True:
            # copy the encoder layers
            temp = copy.deepcopy(layers[0:self.enc_layers*2])
            # add the bottleneck
            temp.append(nn.Linear(sizes[-1], sizes[-1]).to(self.device))
            temp.append(nn.ReLU(inplace=False).to(self.device))
            # add the old layers of the decoder
            temp += layers[self.enc_layers*2:]
            layers = copy.deepcopy(temp)

        self.net = nn.Sequential(*layers)

        print(self.net)

    def forward(self, x):
        return self.net(x)

    """
        Function that resets the parameter weights of 
        the layers in a neural network. The layers
        in our case are all fully connected (torch.nn.Linear)

        Params:
            m (torch.nn.Module): the layer to reset weights
    """
    def weight_reset(self, m):
        if isinstance(m, torch.nn.Linear):
            m.reset_parameters()


##SAMPLING

samplers

In [0]:
import torch 
import numpy as np

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'

"""
    Class that performs the weighted sampling
    based on the errors generated by the ensemble
    components of BAE
"""
class WeightedSampler:

    
    def __init__(self, X):
        """
        Parameters:
            - X (torch.tensor): The original 
                    input tensor
        """
        self.X = X

    """
        This function takes a subset of the entire
        tensor X and returns that subsample. The
        subsample is selected with replacement.

        Parameters:
            - errors (numpy.array): The rec errors 
                    of iteration it
            - it (int): The iteration of the training
                    phase
        return torch.tensor representing the subsample
            of the original data
    """
    def sample(self, errors, it):
        # select all the instances in the first
        # iterations
        if it == 0:
            return self.X
        else:
            # calculate the weights as described in
            # the paper
            weights = (1 / errors)/np.sum(1 / errors)
            # sample the indices with replacement
            indices = np.random.choice(list(range(\
                len(errors))), self.X.shape[0], p=weights)
            # return the subsample
            return self.X[indices]


##STOPPAGE

stoppers

In [0]:
import numpy as np
from numpy import linalg as la
import torch

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'

class NormDiffStopper:
    def __init__(self, precision, norm_kind=np.inf):
        self.norm_kind = norm_kind
        self.precision = precision

    def stop(self, prev, curr, it):
        dist = torch.abs(prev - curr)
        return it >= 1 and la.norm(dist, self.norm_kind)/len(dist) <= self.precision

class FixedStopper:
    def __init__(self, maxit):
        self.maxit = maxit


    def stop(self, prev, curr, it):
        return it  >= self.maxit


class LossValueStopper:

    def __init__(self, threshold):
        self.threshold = threshold

    def stop(self, prev, curr, it):
        return it > 1 and abs(prev - curr) < self.threshold


##UTILS

data

In [0]:
import numpy as np
import pandas as pd
import torch
from scipy.io.arff import loadarff
from scipy.io.matlab import loadmat
from sklearn import preprocessing

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'


class DataLoader:

    def __init__(self, filename, normalise=True,\
        b=-1.0, f=1.0):

        self.filename = filename
        self.normalise = normalise
        self.b = b
        self.f = f
       

    def __data_init(self):
        try:
            data = None

            if self.filename.endswith('.arff'):
                # load the arff file
                data = loadarff(self.filename)

                # transform the arff data into a pandas dataframe
                data = pd.DataFrame(data[0])

            elif self.filename.endswith('.csv'):
                data = pd.read_csv(self.filename)

            elif self.filename.endswith('.mat'):
                data = loadmat(self.filename)

                data = pd.DataFrame(np.hstack((data['X'],\
                    data['y'])))

            # if the column 'id' is in the column list of the data,
            # then remove it
            if 'id' in data.columns:
                data = data.drop(labels='id', axis=1)

            dummies = pd.get_dummies(data[data.columns[-1]], drop_first=True)
            data = data.drop(data.columns[-1], axis = 1)
            data = data.join(dummies)

            data = data.rename(columns={\
                data.columns[-1]: 'outlier'})

            if self.normalise == True:
                x = data.loc[:, data.columns != 'outlier'] 
                # scale column-wise data in [self.b, self.f]
                # add the outlier column to the scaled data
                data = self.__scale_data(x,\
                        self.b, self.f).join(data['outlier'])
            
            return data
        except NotImplementedError:
            print('The file %s contains not supported attributes.'\
                %self.filename)

    def __scale_data(self, X, b, f, min=0, max=255, opt=True):
        if opt == True:
            max = X.max(axis=0)
            min = X.min(axis=0)

        diff = max - min

        X = (f-b) * (X-min)/diff + b

        # case in which the minimium and maximum of a column
        # are the same
        X.dropna(axis='columns')
        X = X.fillna((f+b)/2)

        return X


    def prepare_data(self):
        data = self.__data_init()
        # get the ground truth labels
        y = data['outlier'].values

        if 'id' in data.columns:
            X = data.drop(labels='id', axis=1)
        else:
            X = data
        X = X.drop(labels='outlier', axis=1)

        # normalise the data to [0,1]
        X = torch.tensor(X.values, dtype=torch.float32)

        print("Input tensor shape {0}".format(X.shape))
        print("Ground truth tensor shape {0}".format(y.shape))


        return X, y


dump

In [0]:
import gzip
import json
import os
import uuid

import numpy as np
import pandas as pd
import torch

__author__ = 'Hamed Sarvari, Bardh Prenkaj, Giovanni Stilo'

NS_AN_SCORES = "scores"
NS_REC_ERROR_IT = "rec-error-it"
NS_RANKING =  "ranking"
NS_WEIGHTS =  "weights"
NS_LOSS = "loss-value"
SEP =','

class MetaInfo:

    def __init__(self, params):

        self.expid = uuid.uuid1().hex
        self.params = params          
        self.params['eid']=self.expid

class ExperimentDumper:
    
    def __init__(self, meta_info, path="../resources/traces"):

        self.path=path
        self.meta_info=meta_info
        
        self.namespaces = dict()

        dirname = os.path.join(self.path,\
            self.meta_info.params['group'],\
                self.meta_info.expid)

        self.meta_info.params['trace-path'] = dirname

        os.makedirs(dirname)
        
        with open(os.path.join(self.path,\
            self.meta_info.expid \
                + '.json'), 'w') as outfile:  
            json.dump(self.meta_info.params, outfile)

    def trace_array(self,namespace,data):

        self.get_writer(namespace)\
            .write(SEP.join([str(i) for i in data])\
                + '\n')

    def trace_model(self, model, it):
        
        torch.save(model, os.path.join(self.path,\
             self.meta_info.params['group'],\
                 self.meta_info.expid,\
                     model.__class__.__name__\
                         + "@" + it + ".pth"))

    def close(self):
        for k in self.namespaces.keys():
            self.namespaces[k].close()
        
    def get_writer(self, namespace):

        if self.namespaces.get(namespace,None) is None:

                self.namespaces[namespace] = gzip.open(\
                    os.path.join(self.path,\
                    self.meta_info.params['group'],\
                        self.meta_info.expid, namespace\
                            + ".gz"), "wt")
                self.namespaces[namespace].write('# '\
                + namespace + ' for :'\
                    + json.dumps(self.meta_info.params)\
                        + '\n')
        
        return self.namespaces[namespace]
        

class LoadDump:

    def __init__(self, path='../resources/traces'):

        self.path = path
        
        self.descriptors = [f for f in \
            os.listdir(self.path) \
                if os.path.isfile(os.path\
                    .join(self.path, f))]


    def filter(self, query):
        results = []
        for descriptor in self.descriptors:
            data = json.load(open(os\
                .path.join(self.path,\
                    descriptor), 'r'))

            if all(item in data.items()\
                for item in query.items()):

                results.append(\
                    data.get('trace-path',None))

        return results

    def load(self, trace, doc=NS_REC_ERROR_IT):

        try:
            with gzip.open(os.path.join(trace,\
                doc + '.gz'), "r") as f:

                res = pd.read_csv(f,\
                    skiprows=1, header=None).values

                res = np.array(res, dtype=np.double)

            return res
        except:
            return None


##MAIN

main

In [0]:
from __future__ import print_function

import collections

import numpy as np
import torch

import utils.dump as dumper
from config import conf_sample as conf
from models.fcae import FullyConnectedAE
from sampling.samplers import WeightedSampler
from utils.data import DataLoader
from stoppage.stoppers import LossValueStopper

__author__ = 'bardhp95'


def get_model(enc_layers, dim):

    model = FullyConnectedAE(enc_layers, dim,\
        conf.shrinker, bottleneck=conf.BOTTLENECK)

    opt = torch.optim.Adam(model.parameters(),\
        lr=conf.LR,\
        amsgrad=True, weight_decay=conf.WEIGHT_DECAY)

    return model, opt

"""
    Function that resets the reconstruction
    errors to 0 after each fold is done
    executing

    Parameters:
        - dim (int): represents the
                dimensionality of the data
                points

    returns numpy array of zeros
"""
def reset_errors(dim):
    return np.array([0 for i in range(dim)])

"""
    Function which gets the reconstruction errors
    for each data point

    Parameters:
        - model (?): the trained torch model
        - X (torch.tensor): the original data points
    
    returns numpy.array with the reconstruction errors
        of each data point
"""
def get_error(model, X):
    eval_err = torch.nn.MSELoss()
    return np.array([eval_err(model(X[i]),\
        X[i]).data.item() for i in range(X.shape[0])])

def shift_losses(losses, next_loss):
    losses[0] = losses[-1]
    losses[-1] = next_loss
    return losses

# define the training stopper
stopper = LossValueStopper(conf.STOP_TH)
# iterate the list of datasets
for dataset in conf.DATASETS:
    ############### Dataset loader ######################
    loader = DataLoader(conf.DPATH + dataset,\
        normalise=conf.NORM, b=conf.B_NORM,\
            f=conf.E_NORM)

    X, y = loader.prepare_data()

    sampler = WeightedSampler(X)

    for enc_layer in range(1,conf.MAX_ENC_LAYERS):
        # what's the number of layers when taking in input
        # the number of encoder layers? It depends if
        # we're considering a single bottleneck or not
        num_layers = enc_layer*2+1 if conf.BOTTLENECK else enc_layer*2
        print(f'Training model with {num_layers} layers...')
        # get the model with the specified number
        # of encoder layers and the data
        # dimensionality
        model, opt = get_model(enc_layer, X.shape[1])
        # perform experiments for the
        # specified number of folds
        for fold in range(conf.FOLDS):
            rankings = list()
            # reset the training data indices
            rec_errors = reset_errors(X.shape[0])

            print(f'Dataset {dataset} fold {fold+1}')
            # Dump and loader characteristics
            params = dict()
            params['optimiser'] = opt.__class__.__name__
            params['group'] = model.__class__.__name__
            params['dataset'] = dataset
            params['run'] = fold + 1
            params['loss'] = conf.loss.__class__.__name__
            params['normaliser'] = None
            params['stopper'] = None
            params['layer_num'] = num_layers

            logger = dumper.ExperimentDumper(dumper\
                .MetaInfo(params))

            # repeat the training for a certain amount 
            # of components
            for it in range(conf.ITERATIONS):
                # initialize the previous and current 
                # training losses
                # losses[0] -> previous loss
                # losses[1] -> current loss
                losses = [-1,-1]
                # reset the model's weights for the
                # next AE component
                model.apply(model.weight_reset)
                # sample the virtual dataset 
                # \mathbf{X}^{(i)}
                # as described in the paper 
                X_i = sampler.sample(rec_errors, it)
                # tell torch that the virtual dataset 
                # doesn't need to compute the gradients
                X_i.requires_grad_(False)
                
                print(f'Training component {it+1}')
                # train the components for a certain
                # number of epochs
                for epoch in range(conf.EPOCHS):
                    model.train()
                    # get the reonstructed output
                    # of the virtual dataset 
                    # \mathbf{X}^{(i)}
                    X_i_out = model(X_i)
                    # get the loss according to the 
                    # MSE criterion
                    loss = conf.loss(X_i_out, X_i)

                    losses = shift_losses(losses, loss)
                    # zero the old gradients for the 
                    # backpropagation phase
                    opt.zero_grad()
                    # compute the backpropagation graph
                    loss.backward()
                    # update the weights of the ADAM 
                    # optimiser
                    opt.step()

                    if (epoch + 1) % 50 == 0:
                        print(f'Epoch {epoch+1} - loss: {loss.item()}')

                    if stopper.stop(losses[0], losses[1], epoch):
                        print(f'Loss difference {abs(losses[0]-losses[1])} < {conf.STOP_TH}')
                        print(f'Breaking the training of iteration {it+1} at epoch {epoch+1}')
                        break
                # evaluate the reconstruction errors of 
                # the trained
                # model in the original datasetd
                rec_errors = get_error(model, X)

                print(f'Finished training component {it+1}')
                print(f'Component {it+1}: loss {loss.item()} sum of rec_errors {np.sum(rec_errors)}')
                # append the reconstruction errors in 
                # the ranking list
                # this is used to compute the consensus 
                # function
                rankings.append(rec_errors)
                # trace reconstruction errors for each
                # iteration
                logger.trace_array(dumper\
                    .NS_REC_ERROR_IT, rec_errors)
                # trace the loss of each iteration
                logger.trace_array(dumper.NS_LOSS,\
                    [loss.data.numpy()])
                # trace the model for each iteration
                logger.trace_model(model, str(it + 1))

            # combine the overall ranking
            final_ranking = conf.consensus(rankings)
            # log the overall ranking
            logger.trace_array(dumper.NS_RANKING,\
                final_ranking)

    print(f'Finished training BAE with a base architecture of {num_layers} layers.')
